In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import csv
import requests
import time
import os
import json

In [ ]:
# configure
file_path = '../data/raw/save_answers/questions.txt'
output_path = '../data/load/save_answers'
delay_seconds = 10

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
list_of_questions = []

with open(file_path, 'rt') as txt_file:
  for text in txt_file:
    text = text.strip()
    if text != '':
        list_of_questions.append(text)

In [ ]:
file_data = {}
file_extra = {}

for question in list_of_questions:
    list_data = []
    list_data_extra = []
    time.sleep(delay_seconds)
    response = requests.get('https://scripturecentralqa.org/api/search', params={'q': question})
    
    res = response.content.decode('utf-8')
    result_data = json.loads(res)
    for item in result_data['results']:
        result_temp = json.dumps(item)
        list_data.append(result_temp)
        if len(list_data) > 4:
            break

    for item in result_data['results']:
        result_temp = json.dumps(item)
        list_data_extra.append(result_temp)
        if len(list_data_extra) > 19:
            break

    result_set = '\n--------\n'.join(list_data)

    file_data[question] = [result_set,result_data['answer']]
    file_extra[question] = [list_data_extra,result_data['answer']]

In [ ]:
with open(output_path + '/question-answers.csv', 'a') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Question', 'Results', 'Answer'])  # header row
    
for key,value in file_data.items():
  with open(output_path + '/question-answers.csv', 'a') as csvfile:
    writer = csv.writer(csvfile)
    # for each question, the file should contain a line like this:
    writer.writerow([key, value[0], value[1]])

In [ ]:
with open(output_path + '/rank-question-answers.csv', 'a') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Question', 'ResultRank', 'ResultURL', 'ResultTitle', 'ResultText'])  # header row
    
for key,value in file_extra.items():
  with open(output_path + '/rank-question-answers.csv', 'a') as csvfile:
    writer = csv.writer(csvfile)
   
    for val in value[0]:

      value_res = json.loads(val)
     
      # for each question, the file should contain a line like this:
      writer.writerow([key, value_res["index"], value_res["url"], value_res["title"], value_res["text"]])

print('End')